### Scrape Schedule

In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Scrape the NFL schedule for the current season
url = "https://www.pro-football-reference.com/years/2025/games.htm"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

# Find the schedule table
table = soup.find("table", id="games")

# Extract headers
headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]

# Extract rows
rows = []
for row in table.find("tbody").find_all("tr"):
    if "class" in row.attrs and "thead" in row["class"]:
        continue  # skip mid-table headers
    cells = [cell.get_text(strip=True) for cell in row.find_all(["th", "td"])]
    rows.append(cells)

# Create DataFrame
sked_df = pd.DataFrame(rows, columns=headers)
sked_df.columns = ['Week', 'Day', 'Date', 'Time', 'Visitor', 'H/A', 'Home', 'Boxscore', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL']
sked_df = sked_df[['Week', 'Day', 'Date', 'Time', 'Visitor', 'H/A', 'Home']]
sked_df['Matchup String'] = sked_df['Visitor'] + " " + sked_df['H/A'].apply(lambda x: "vs" if x == "" else "@") + " " + sked_df['Home']
sked_df['Start Time'] = sked_df['Day'] + ' ' + sked_df['Time']
# roll up the start time column into buckets: Thursday, Sunday Morning, Sunday Early Afternoon, Sunday Late Afternoon, Sunday Night, Monday Night... I don't care about Friday or Saturday
def bucket_start_time(start_time):
    if "Thu" in start_time:
        return "Thursday"
    elif "Mon" in start_time:
        return "Monday Night"
    elif "Sun" in start_time:
        time_part = start_time.split(' ')[-1]
        hour = int(time_part.split(':')[0])
        if hour < 12 and time_part.endswith('AM'):
            return "Sunday Morning"
        elif 1 <= hour < 4 and time_part.endswith('PM'):
            return "Sunday Early Afternoon"
        elif 4 <= hour < 8 and time_part.endswith('PM'):
            return "Sunday Late Afternoon"
        else:
            return "Sunday Night"
    else:
        return "Other"
sked_df['Week'] = sked_df['Week'].astype(int)
sked_df['Start Time Bucket'] = sked_df['Start Time'].apply(bucket_start_time)
sked_df

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
0,1,Thu,2025-09-04,8:20PM,Philadelphia Eagles,,Dallas Cowboys,Philadelphia Eagles vs Dallas Cowboys,Thu 8:20PM,Thursday
1,1,Fri,2025-09-05,8:00PM,Los Angeles Chargers,,Kansas City Chiefs,Los Angeles Chargers vs Kansas City Chiefs,Fri 8:00PM,Other
2,1,Sun,2025-09-07,1:00PM,Pittsburgh Steelers,@,New York Jets,Pittsburgh Steelers @ New York Jets,Sun 1:00PM,Sunday Early Afternoon
3,1,Sun,2025-09-07,1:00PM,Washington Commanders,,New York Giants,Washington Commanders vs New York Giants,Sun 1:00PM,Sunday Early Afternoon
4,1,Sun,2025-09-07,1:00PM,Las Vegas Raiders,@,New England Patriots,Las Vegas Raiders @ New England Patriots,Sun 1:00PM,Sunday Early Afternoon
...,...,...,...,...,...,...,...,...,...,...
267,18,Sun,2026-01-04,1:00PM,Tennessee Titans,@,Jacksonville Jaguars,Tennessee Titans @ Jacksonville Jaguars,Sun 1:00PM,Sunday Early Afternoon
268,18,Sun,2026-01-04,1:00PM,Baltimore Ravens,@,Pittsburgh Steelers,Baltimore Ravens @ Pittsburgh Steelers,Sun 1:00PM,Sunday Early Afternoon
269,18,Sun,2026-01-04,1:00PM,Los Angeles Chargers,@,Denver Broncos,Los Angeles Chargers @ Denver Broncos,Sun 1:00PM,Sunday Early Afternoon
270,18,Sun,2026-01-04,1:00PM,Seattle Seahawks,@,San Francisco 49ers,Seattle Seahawks @ San Francisco 49ers,Sun 1:00PM,Sunday Early Afternoon


In [40]:
pivoted = pd.pivot_table(sked_df, index='Start Time Bucket', columns='Week', values='Matchup String', aggfunc='count').fillna(0).astype(int)
pivoted = pivoted.reindex(index=["Thursday", "Sunday Morning", "Sunday Early Afternoon", "Sunday Late Afternoon", "Sunday Night", "Monday Night", "Other"])
pivoted.index.name = None
pivoted.style.background_gradient(axis=1)

Week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Thursday,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,3,0
Sunday Morning,0,0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0
Sunday Early Afternoon,8,9,9,7,6,7,6,7,8,7,7,7,7,8,8,7,10,16
Sunday Late Afternoon,4,3,4,4,4,3,4,3,3,3,4,4,3,3,5,4,1,0
Sunday Night,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
Monday Night,1,2,1,2,1,2,2,1,1,1,1,1,1,1,1,1,1,0
Other,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0


After bucketing the data, we can see at a glance how the games are distributed across the start time windows each week. 

In [45]:
sked_df[(sked_df['Start Time Bucket'] == 'Thursday') & (sked_df['Week'].isin([13, 17]))]

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
178,13,Thu,2025-11-27,1:00PM,Green Bay Packers,@,Detroit Lions,Green Bay Packers @ Detroit Lions,Thu 1:00PM,Thursday
179,13,Thu,2025-11-27,4:30PM,Kansas City Chiefs,@,Dallas Cowboys,Kansas City Chiefs @ Dallas Cowboys,Thu 4:30PM,Thursday
180,13,Thu,2025-11-27,8:20PM,Cincinnati Bengals,@,Baltimore Ravens,Cincinnati Bengals @ Baltimore Ravens,Thu 8:20PM,Thursday
240,17,Thu,2025-12-25,1:00PM,Dallas Cowboys,@,Washington Commanders,Dallas Cowboys @ Washington Commanders,Thu 1:00PM,Thursday
241,17,Thu,2025-12-25,4:30PM,Detroit Lions,@,Minnesota Vikings,Detroit Lions @ Minnesota Vikings,Thu 4:30PM,Thursday
242,17,Thu,2025-12-25,8:15PM,Denver Broncos,@,Kansas City Chiefs,Denver Broncos @ Kansas City Chiefs,Thu 8:15PM,Thursday


We can quickly observe that there is usually only 1 game on Thursday night, with the exception of Weeks 13 and 17, which feature 3 Thursday night games. Week 13 happens to be Thanksgiving, and Week 17 is Christmas, so that checks out!

In [42]:
sked_df[sked_df['Start Time Bucket'] == 'Sunday Morning']

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
49,4,Sun,2025-09-28,9:30AM,Pittsburgh Steelers,,Minnesota Vikings,Pittsburgh Steelers vs Minnesota Vikings,Sun 9:30AM,Sunday Morning
65,5,Sun,2025-10-05,9:30AM,Minnesota Vikings,@,Cleveland Browns,Minnesota Vikings @ Cleveland Browns,Sun 9:30AM,Sunday Morning
79,6,Sun,2025-10-12,9:30AM,Denver Broncos,@,New York Jets,Denver Broncos @ New York Jets,Sun 9:30AM,Sunday Morning
94,7,Sun,2025-10-19,9:30AM,Los Angeles Rams,@,Jacksonville Jaguars,Los Angeles Rams @ Jacksonville Jaguars,Sun 9:30AM,Sunday Morning
136,10,Sun,2025-11-09,9:30AM,Atlanta Falcons,@,Indianapolis Colts,Atlanta Falcons @ Indianapolis Colts,Sun 9:30AM,Sunday Morning
150,11,Sun,2025-11-16,9:30AM,Washington Commanders,@,Miami Dolphins,Washington Commanders @ Miami Dolphins,Sun 9:30AM,Sunday Morning


Turning our attention to the rare Sunday morning games, we have 6 of them total. All of these games are Europe games, which is exciting. The NFL is globalizing American Football! But perhaps burdensome for fans, these games are at 9:30am, so fantasy managers have to remember to set their lineups earlier than usual.

Their global games are as follows:

| Week | Matchup | Location |
|:---:|:---:|:---:|
| Week 1 | LAC @ KC | Sao Paolo, Brazil |
| Week 4 | PIT @ MIN | Dublin, Ireland |
| Week 5 | MIN @ CLE | London, England |
| Week 6 | DEN @ NYJ | London, England |
| Week 7 | LAR @ JAX | London, England |
| Week 10 | ATL @ IND | Berlin, Germany |
| Week 11 | WAS @ MIA | Madrid, Spain |

In [46]:
sked_df[sked_df['Start Time Bucket'] == 'Other']

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
1,1,Fri,2025-09-05,8:00PM,Los Angeles Chargers,,Kansas City Chiefs,Los Angeles Chargers vs Kansas City Chiefs,Fri 8:00PM,Other
181,13,Fri,2025-11-28,3:00PM,Chicago Bears,@,Philadelphia Eagles,Chicago Bears @ Philadelphia Eagles,Fri 3:00PM,Other
225,16,Sat,2025-12-20,1:00PM,Green Bay Packers,@,Chicago Bears,Green Bay Packers @ Chicago Bears,Sat 1:00PM,Other
226,16,Sat,2025-12-20,1:00PM,Philadelphia Eagles,@,Washington Commanders,Philadelphia Eagles @ Washington Commanders,Sat 1:00PM,Other


Among the games labeled "Other," we have Friday games and Saturday games. The first Friday game shown in the dataframe above is the Dublin game mentioned earlier. The other Friday game is a Black Friday game during the afternoon (*Shameless plug: the final night of NBA Cup Group Play is at night on Black Friday*).

The two Saturday games? I honestly have no idea why the NFL is doing those. I guess I'll find out soon enough.

My real reason for investigating these weekly schedule start time quirks is because I wanted to understand why the f--- the NFL is scheduling 9 games in the 1pm window. I can barely handle Scott Hanson's Octobox as it is... the Brady Bunch view is just too much for me, so there's gotta be good reason.

In [48]:
sked_df[sked_df['Week'] == 2]

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
16,2,Thu,2025-09-11,8:15PM,Green Bay Packers,,Washington Commanders,Green Bay Packers vs Washington Commanders,Thu 8:15PM,Thursday
17,2,Sun,2025-09-14,1:00PM,New England Patriots,@,Miami Dolphins,New England Patriots @ Miami Dolphins,Sun 1:00PM,Sunday Early Afternoon
18,2,Sun,2025-09-14,1:00PM,Dallas Cowboys,,New York Giants,Dallas Cowboys vs New York Giants,Sun 1:00PM,Sunday Early Afternoon
19,2,Sun,2025-09-14,1:00PM,Detroit Lions,,Chicago Bears,Detroit Lions vs Chicago Bears,Sun 1:00PM,Sunday Early Afternoon
20,2,Sun,2025-09-14,1:00PM,Buffalo Bills,@,New York Jets,Buffalo Bills @ New York Jets,Sun 1:00PM,Sunday Early Afternoon
21,2,Sun,2025-09-14,1:00PM,San Francisco 49ers,@,New Orleans Saints,San Francisco 49ers @ New Orleans Saints,Sun 1:00PM,Sunday Early Afternoon
22,2,Sun,2025-09-14,1:00PM,Baltimore Ravens,,Cleveland Browns,Baltimore Ravens vs Cleveland Browns,Sun 1:00PM,Sunday Early Afternoon
23,2,Sun,2025-09-14,1:00PM,Los Angeles Rams,@,Tennessee Titans,Los Angeles Rams @ Tennessee Titans,Sun 1:00PM,Sunday Early Afternoon
24,2,Sun,2025-09-14,1:00PM,Cincinnati Bengals,,Jacksonville Jaguars,Cincinnati Bengals vs Jacksonville Jaguars,Sun 1:00PM,Sunday Early Afternoon
25,2,Sun,2025-09-14,1:00PM,Seattle Seahawks,@,Pittsburgh Steelers,Seattle Seahawks @ Pittsburgh Steelers,Sun 1:00PM,Sunday Early Afternoon


In Week 2, the reason seems somewhat intuitive. The NFL scheduled the Philadelphia Eagles at the Kansas City Chiefs, the rematch of last year's Super Bowl. I imagine they wanted to maximize viewership for that game by minimizing the number of backup games. I also imagine they scheduled it early in the season to make sure that both teams were off to mitigate the risk of one or both teams being off to a bad start or plagued by injuries. Notably, Week 2 also already features two Monday Night Football games, so they really couldn't have distributed the games better here given that they were trying to get max eyeballs on the Eagles vs. Chiefs game.

In [49]:
sked_df[sked_df['Week'] == 3]

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
32,3,Thu,2025-09-18,8:15PM,Buffalo Bills,,Miami Dolphins,Buffalo Bills vs Miami Dolphins,Thu 8:15PM,Thursday
33,3,Sun,2025-09-21,1:00PM,Washington Commanders,,Las Vegas Raiders,Washington Commanders vs Las Vegas Raiders,Sun 1:00PM,Sunday Early Afternoon
34,3,Sun,2025-09-21,1:00PM,Philadelphia Eagles,,Los Angeles Rams,Philadelphia Eagles vs Los Angeles Rams,Sun 1:00PM,Sunday Early Afternoon
35,3,Sun,2025-09-21,1:00PM,Minnesota Vikings,,Cincinnati Bengals,Minnesota Vikings vs Cincinnati Bengals,Sun 1:00PM,Sunday Early Afternoon
36,3,Sun,2025-09-21,1:00PM,Jacksonville Jaguars,,Houston Texans,Jacksonville Jaguars vs Houston Texans,Sun 1:00PM,Sunday Early Afternoon
37,3,Sun,2025-09-21,1:00PM,Tampa Bay Buccaneers,,New York Jets,Tampa Bay Buccaneers vs New York Jets,Sun 1:00PM,Sunday Early Afternoon
38,3,Sun,2025-09-21,1:00PM,Carolina Panthers,,Atlanta Falcons,Carolina Panthers vs Atlanta Falcons,Sun 1:00PM,Sunday Early Afternoon
39,3,Sun,2025-09-21,1:00PM,Pittsburgh Steelers,@,New England Patriots,Pittsburgh Steelers @ New England Patriots,Sun 1:00PM,Sunday Early Afternoon
40,3,Sun,2025-09-21,1:00PM,Indianapolis Colts,@,Tennessee Titans,Indianapolis Colts @ Tennessee Titans,Sun 1:00PM,Sunday Early Afternoon
41,3,Sun,2025-09-21,1:00PM,Cleveland Browns,,Green Bay Packers,Cleveland Browns vs Green Bay Packers,Sun 1:00PM,Sunday Early Afternoon


In Week 3, I don't really understand why they had to schedule 9 games in the Sunday early afternoon slot. The Sunday late afternoon slot features four games––Chargers vs. Broncos, Seahawks vs. Saints, Bears vs. Cowboys, and 49ers vs. Cardinals––none of which are marquee. My hypothesis here is the Monday Night game, Lions at Ravens, was too juicy to be stacked with another simultaneous game, hence why Sunday had to be packed with 14 games.

Sometimes I wonder what NFL Sunday would be like if there were staggered starts... just game after game kicking off 20 minutes apart. Scott Hanson would probably lose his mind.

Well, anyway, I am pleased to report that there are no other instances of more than 8 games in the Sunday early afternoon slot after Week 3 this season. Part of the reason for this is bye weeks take place from Week 5 through Week 14, so there aren't even 16 games to schedule during those weeks. Weeks 17 and 18 do have 9+ games tentatively scheduled for the Sunday early afternoon slot, but based on last year's schedule, I am fairly confident that the timing of those games will be flexed depending on competitive implications.

Hope you found this schedule rabbit hole as interesting as I did (: